In [1]:
clean_up = True
%run stdPackages.ipynb
d['data'] = os.path.join(d['data'], 'IO2018')

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.


# Setup for production module, GR18

The notebook defines the nesting structure for the production module. Additional tweaks to the production data can be loaded here as well. At this stage, we only define the main nesting structure as a mapping ```nestProduction```.

### Settings: 

Load IO data from the general model and store a new one with data specific to the production module:

In [ ]:
name = 'GR18' # name of model
module = name+'_Production' # name of module
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'IO_{name}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)
db = GpyDB(**{'name': module})

NameError: name 'GpyDB' is not defined

### Nesting

**Non-materials nest:**

*Full index:*

In [3]:
mFull = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','KE'), 
                                   ('KE','iM'), ('KE','E'), 
                                   ('E','35011_input'), ('E','35011_F')], names = ['n','nn'])
E = pd.Index(['35011','35011_F'], name = 'n')

Some of the sectors may not use the energy inputs, or only use either the foreign/domestic types:

In [4]:
m_onlyEdom = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','KE'), 
                                   ('KE','iM'), ('KE','35011_input')], names = ['n','nn'])
m_onlyEfor = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','KE'), 
                                   ('KE','iM'), ('KE','35011_F')], names = ['n','nn'])
m_noE = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','iM')], names = ['n','nn'])

Look at energy demand with domestic and foreign types in rows:

In [5]:
Edf = adj.rc_pd(db_IO.get('vD'), E).unstack('n')

If the sector is not in ```Edf```, the sector does not demand any energy inputs. If the foreign electricity good is NaN, the sector only uses domestic energy. If the domestic sector is NaN, they only use foreign:

In [6]:
s_noE = adj.rc_pd(db_IO.get('s_p'), ('not',Edf))
s_onlyFor = adj.rc_pd(db_IO.get('s_p'), Edf[Edf['35011'].isna()])
s_onlyDom = adj.rc_pd(db_IO.get('s_p'), Edf[Edf['35011_F'].isna()])
s_fullE = adj.rc_pd(db_IO.get('s_p'), ('not', ('or', [s_noE, s_onlyFor, s_onlyDom])))

Establish mappings:

In [7]:
m = pd.MultiIndex.from_tuples(np.hstack([pyDatabases.cartesianProductIndex([s_noE, m_noE]).values,
                                         pyDatabases.cartesianProductIndex([s_onlyFor, m_onlyEfor]).values,
                                         pyDatabases.cartesianProductIndex([s_onlyDom, m_onlyEdom]).values,
                                         pyDatabases.cartesianProductIndex([s_fullE, mFull]).values]),
                              names = ['s','n','nn'])

Replace upper-most level with name of sector:

In [8]:
df = m.to_frame(index=False) 
df.loc[df.n == 'KELBM','n'] = df.loc[df.n == 'KELBM', 's']
m = pd.MultiIndex.from_frame(df)

**Materials nest:**

There are quite a lot of sparsity in the materials nest - so we deal explicitly with it.

In [9]:
E = pd.Index(['35011','35011_F'], name = 'n')

For [s,n] in ```dImport[s,n,nn]```, we define a mapping from ```RxE```to an intermediate good ```RxEym_x``` that is again an aggregate of the domestic/foreign goods ```x,x_F```. We do this for any sectors except the energy goods:

In [10]:
E = pd.Index(['35011','35011_F'], name = 'n') # treat these types of good in a different way (not in materials nest)
df = adj.rc_pd(db_IO.get('dImport'), ('and', [db_IO.get('s_p'), ('not',E)]) # imports for all domestic production sectors, not E
              ).to_frame(index=False).assign(RxEym= lambda x: 'RxEym_'+x['n'], RxE = 'RxE', n = lambda x: x['n']+'_input') # add intermediate goods
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','RxEym']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','RxEym','n']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','RxEym','nn']]).rename(['s','n','nn']))

For [s,n] in ```dImport_dom[s,n]```, we define a mapping from ```RxE```to the domestic goods:

In [11]:
df = adj.rc_pd(db_IO.get('dImport_dom'), ('and', [db_IO.get('s_p'), ('not',E)])
              ).to_frame(index=False).assign(RxE = 'RxE', n = lambda x: x['n']+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','n']]).rename(['s','n','nn']))

For [s,n] in ```dImport_for[s,n]```, we define a mapping from ```RxE```to the forein goods:

In [12]:
df = adj.rc_pd(db_IO.get('dImport_for'), ('and', [db_IO.get('s_p'), ('not',E)])
              ).to_frame(index=False).assign(RxE = 'RxE')
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','n']]).rename(['s','n','nn']))

Add to database:

In [13]:
db['nestProduction'] = m 

### Export data:

In [14]:
db.export(repo = d['data'])